In [1]:
import math
import torch
import transformers
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling, TFAutoModelForCausalLM
import evaluate
from datasets import DatasetDict, Dataset
import numpy as np
import random
import os
# import wandb
FINAL_ARTISTS = "beyonce,sia,anitta,adele,eminem,ed_sheeran,coldplay,pink,taylor_swift,imagine_dragons,justin_bieber,ludmilla,the_beatles,maroon_5,bruno_mars,lady_gaga,lana_del_rey,ariana_grande,christina_perri,phil_collins,rihanna,camila_cabello,bon_jovi,elton_john,john_legend,john_lennon,pink_floyd,scorpions,red_hot_chili_peppers,50_cent,nirvana,queen,katy_perry,alok,u2,black_eyed_peas,michael_jackson,jason_mraz,guns_n_roses,alicia_keys,rammstein,shawn_mendes,linkin_park,shakira".split(",")

In [3]:
tokenizer = AutoTokenizer.from_pretrained("distilgpt2", pad_token="[PAD]")
model = AutoModelForCausalLM.from_pretrained("distilgpt2")
tokenizer.pad_token = tokenizer.eos_token


## Tokenize and setup data

In [ ]:
def tokenizer_func(data):
    return tokenizer(data["text"], truncation=True)
    
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

block_size = 128

tokenized_datasets = datasets.map(tokenizer_func, batched=True, remove_columns=["text"], num_proc=4)
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    num_proc=4,
    batch_size=1000,
)

# tokenizer.decode(lm_datasets["train"][1]["input_ids"])

In [ ]:
# data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)
# training_args = TrainingArguments(output_dir="output/ts-testing-training-model",evaluation_strategy="epoch",learning_rate=2e-5,weight_decay=0.01, num_train_epochs=25, report_to='wandb', use_mps_device=True, eval_steps=1, logging_steps=4)
# trainer = Trainer(model=model, args=training_args, train_dataset=lm_datasets['train'], eval_dataset=lm_datasets['test'], data_collator=data_collator)

# trainer.train()

In [9]:
eval_results = trainer.evaluate()

print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

***** Running Evaluation *****
  Num examples = 295
  Batch size = 8
You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


  0%|          | 0/37 [00:00<?, ?it/s]

Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: Currently logged in as: bhavyamuni. Use `wandb login --relogin` to force relogin


Perplexity: 26.96
